# How to use a matrix factorization model to recommend movies for a given user with TensorFlow Recommender

#### **Copyright**
**Author:** [ TensorFlow](https://www.tensorflow.org/recommenders)<br>
**Project Type:** TensorFlow Recommender<br>
**Date created:** 2022/17/10<br>
**Training Dataset** This project uses the public dataset [` MovieLens `](https://grouplens.org/datasets/movielens/100k/) dataset with 100K TFRS

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

# Summary
In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Dataset
This project uses the public dataset [` MovieLens `](https://grouplens.org/datasets/movielens/100k/) dataset with 100K TFRS

### Steps
1. Install and Import TFRS
1. Download and Read the Data
1. Build vocabularies
1. Define a Model
1. Create the retrieval Tasks
1. Fit and Evaluate it
1. Create Predictions for a given user


### Import TFRS

First, install and import TFRS:

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 89 kB 3.7 MB/s 
     |████████████████████████████████| 4.7 MB 4.7 MB/s 


In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [ ]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")

# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

movies = movies.map(lambda x: x["movie_title"])

Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [5]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [6]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [8]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [11]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

Epoch 1/3
25/25 [==============================] - 22s 840ms/step - factorized_top_k/top_1_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0041 - factorized_top_k/top_10_categorical_accuracy: 0.0102 - factorized_top_k/top_50_categorical_accuracy: 0.0737 - factorized_top_k/top_100_categorical_accuracy: 0.1533 - loss: 33201.2618 - regularization_loss: 0.0000e+00 - total_loss: 33201.2618
Epoch 2/3
25/25 [==============================] - 19s 767ms/step - factorized_top_k/top_1_categorical_accuracy: 7.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0083 - factorized_top_k/top_10_categorical_accuracy: 0.0218 - factorized_top_k/top_50_categorical_accuracy: 0.1363 - factorized_top_k/top_100_categorical_accuracy: 0.2558 - loss: 30607.3593 - regularization_loss: 0.0000e+00 - total_loss: 30607.3593
Epoch 3/3
25/25 [==============================] - 19s 762ms/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-04 - factorized_top_k/top_5_categor

### Create Predictions for a given user


In [12]:
# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Just Cause (1995)']
